In [281]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import sys, six

# Import Data Sccience libraries
import pandas as pd
import numpy as np

# Instantiates a client
client = language.LanguageServiceClient()

In [5]:
small_suicide_df = pd.DataFrame.from_csv("sample_suicide_watch_text.csv",encoding = "ISO-8859-1").iloc[0:10,:]

small_suicide_df

In [34]:
def sentiment_text(text):
    """Detects sentiment in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects sentiment in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    sentiment = client.analyze_sentiment(document).document_sentiment
    
    return [sentiment.score,sentiment.magnitude]

In [31]:
def entity_sentiment_text(text):
    """Detects entity sentiment in the provided text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    # Detect and send native Python encoding to receive correct word offsets.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    for entity in result.entities:
        print('Mentions: ')
        print(u'Name: "{}"'.format(entity.name))
        for mention in entity.mentions:
            print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
            print(u'  Content : {}'.format(mention.text.content))
            print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
            print(u'  Sentiment : {}'.format(mention.sentiment.score))
            print(u'  Type : {}'.format(mention.type))
        print(u'Salience: {}'.format(entity.salience))
        print(u'Sentiment: {}\n'.format(entity.sentiment))

In [29]:
entity_sentiment_text(record)

Mentions: 
Name: "words"
  Begin Offset : 57
  Content : words
  Magnitude : 0.0
  Sentiment : 0.0
  Type : 2
Salience: 0.3393266499042511
Sentiment: 

Mentions: 
Name: "misconception"
  Begin Offset : 199
  Content : misconception
  Magnitude : 0.800000011920929
  Sentiment : -0.800000011920929
  Type : 2
Salience: 0.19303403794765472
Sentiment: magnitude: 1.600000023841858
score: -0.800000011920929


Mentions: 
Name: "anyone"
  Begin Offset : 330
  Content : anyone
  Magnitude : 0.30000001192092896
  Sentiment : -0.30000001192092896
  Type : 2
Salience: 0.09136639535427094
Sentiment: magnitude: 0.699999988079071
score: -0.30000001192092896


Mentions: 
Name: "attention"
  Begin Offset : 144
  Content : attention
  Magnitude : 0.0
  Sentiment : 0.0
  Type : 2
Salience: 0.09088879823684692
Sentiment: 

Mentions: 
Name: "pain"
  Begin Offset : 218
  Content : pain
  Magnitude : 0.20000000298023224
  Sentiment : -0.20000000298023224
  Type : 2
Salience: 0.05292052775621414
Sentiment: mag

In [59]:
extra_data = []

for record in small_suicide_df.iterrows():
    text = record[1]["text"]
    
    extra_record = sentiment_text(text)
    extra_data.append(extra_record)

In [72]:
extra_data_df = pd.DataFrame(extra_data, columns=["Score","Magnitude"],index=small_suicide_df.index)

extra_data_df

In [70]:
small_combined_df = small_suicide_df.join(extra_data_df)

small_combined_df

In [108]:
depression_df = pd.DataFrame.from_csv("depressionTrainingSet3.csv")

In [119]:
dep500_df = depression_df.iloc[0:500,0:1].reset_index()

dep500_extra = []

for record in dep500_df.iterrows():
    text = record[1]["text"]
    
    extra_record = sentiment_text(text)
    dep500_extra.append(extra_record)

dep500_extra_df = pd.DataFrame(dep500_extra,columns=["sentiment","magnitude"])

dep500_extra_df.shape

In [135]:
big_dep500_df = dep500_df.join(dep500_extra_df)

In [137]:
df = big_dep500_df

In [161]:
nodepression_df = pd.DataFrame.from_csv("depressionTrainingSet4.csv")
nodep500_df = nodepression_df.iloc[0:500,0:1].reset_index()

nodep500_extra = []
print(nodep500_df.shape)

(500, 2)


In [162]:
for record in nodep500_df.iterrows():
    text = record[1]["text"][0:999]
    
    extra_record = sentiment_text(text)
    nodep500_extra.append(extra_record)

nodep500_extra_df = pd.DataFrame(nodep500_extra,columns=["sentiment","magnitude"])

nodep500_extra_df.shape

(500, 2)

In [164]:
all_nodep500 = nodep500_df.join(nodep500_extra_df)

training_data = pd.concat([big_dep500_df, all_nodep500])

training_data.to_csv("training_data.csv")

In [150]:
# Start doing classification with SKlearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [174]:
X = training_data.loc[:,["sentiment","magnitude"]]
y = training_data.loc[:,"risk_of_suicide"].astype(int)

In [199]:
regr = LogisticRegression()

score = cross_val_score(regr,X,y)
mse = cross_val_score(regr,X,y,scoring="neg_mean_squared_error")
print(f"Logistic Regression:\nScore: {score.mean():.3f}\tMSE:{mse.mean():.3f}")

Logistic Regression:
Score: 0.780	MSE:-0.220


In [191]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

In [198]:
for depth in range(2,26):
    boosted_dtc = AdaBoostClassifier(DecisionTreeClassifier(max_depth=depth))
    boosted_dtc.fit(X,y)
    currScore = cross_val_score(boosted_dtc,X,y)
    currMSE = cross_val_score(boosted_dtc,X,y,scoring="neg_mean_squared_error")
    print(f"B-DTC with max_depth {depth:02d}:\tScore: {currScore.mean():.3f}\tMSE: {currMSE.mean():.3f}")

B-DTC with max_depth 02:	Score:0.725	MSE:-0.275
B-DTC with max_depth 03:	Score:0.707	MSE:-0.293
B-DTC with max_depth 04:	Score:0.699	MSE:-0.302
B-DTC with max_depth 05:	Score:0.705	MSE:-0.290
B-DTC with max_depth 06:	Score:0.706	MSE:-0.312
B-DTC with max_depth 07:	Score:0.702	MSE:-0.301
B-DTC with max_depth 08:	Score:0.705	MSE:-0.297
B-DTC with max_depth 09:	Score:0.690	MSE:-0.300
B-DTC with max_depth 10:	Score:0.698	MSE:-0.311
B-DTC with max_depth 11:	Score:0.708	MSE:-0.296
B-DTC with max_depth 12:	Score:0.700	MSE:-0.301
B-DTC with max_depth 13:	Score:0.699	MSE:-0.292
B-DTC with max_depth 14:	Score:0.700	MSE:-0.298
B-DTC with max_depth 15:	Score:0.704	MSE:-0.297
B-DTC with max_depth 16:	Score:0.700	MSE:-0.302
B-DTC with max_depth 17:	Score:0.697	MSE:-0.289
B-DTC with max_depth 18:	Score:0.698	MSE:-0.292
B-DTC with max_depth 19:	Score:0.707	MSE:-0.304
B-DTC with max_depth 20:	Score:0.700	MSE:-0.283
B-DTC with max_depth 21:	Score:0.711	MSE:-0.291
B-DTC with max_depth 22:	Score:0.719	MSE

In [243]:
def get_entity_sentiment(text):
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
            content=text.encode('utf-8'),
            type=enums.Document.Type.PLAIN_TEXT)

    # Detect and send native Python encoding to receive correct word offsets.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    salience_ls = [[] for x in range(8)]
    score_ls = [[] for x in range(8)]
    magnitude_ls = [[] for x in range(8)]

    for entity in result.entities:
        salience_ls[entity.type].append(entity.salience)
        score_ls[entity.type].append(entity.sentiment.score)
        magnitude_ls[entity.type].append(entity.sentiment.magnitude)

    mean_salience_ls = [sum(x)/len(x) if x else 0 for x in salience_ls]
    mean_score_ls = [sum(x)/len(x) if x else 0 for x in score_ls]
    mean_magnitude_ls = [sum(x)/len(x) if x else 0 for x in score_ls]

    all_ls = mean_salience_ls + mean_score_ls + mean_magnitude_ls
    return all_ls

In [252]:
extra_training_data = []
for record in training_data.iterrows():
    text = record[1]["text"][0:999]
    
    extra_record = get_entity_sentiment(text)
    extra_training_data.append(extra_record)

In [253]:
type_ls = ["UNKNOWN",
      "PERSON",
      "LOCATION",
      "ORGANIZATION",
      "EVENT",
      "WORK_OF_ART",
      "CONSUMER_GOOD",
      "OTHER"
     ]

ls = ["salience","sentiment","magnitude"]

together = []

for x in ls:
    for y in type_ls:
        together.append(y+"_"+x)

extra_training_data_df = pd.DataFrame(extra_training_data,columns=together)

In [259]:
training_data.index = extra_training_data_df.index

In [261]:
extended_training_data = training_data.join(extra_training_data_df)
extended_training_data.shape

(1000, 28)

In [268]:
# Should yield better performance

X = extended_training_data.loc[:,"sentiment":]
y = extended_training_data.loc[:,"risk_of_suicide"].astype(int)

In [270]:
for depth in range(2,26):
    boosted_dtc = AdaBoostClassifier(DecisionTreeClassifier(max_depth=depth))
    currScore = cross_val_score(boosted_dtc,X,y)
    currMSE = cross_val_score(boosted_dtc,X,y,scoring="neg_mean_squared_error")
    print(f"B-DTC with max_depth {depth:02d}:\tScore: {currScore.mean():.3f}\tMSE: {currMSE.mean():.3f}")

B-DTC with max_depth 02:	Score: 0.746	MSE: -0.253
B-DTC with max_depth 03:	Score: 0.734	MSE: -0.270
B-DTC with max_depth 04:	Score: 0.780	MSE: -0.240
B-DTC with max_depth 05:	Score: 0.752	MSE: -0.239
B-DTC with max_depth 06:	Score: 0.763	MSE: -0.238
B-DTC with max_depth 07:	Score: 0.753	MSE: -0.241
B-DTC with max_depth 08:	Score: 0.778	MSE: -0.247
B-DTC with max_depth 09:	Score: 0.751	MSE: -0.253
B-DTC with max_depth 10:	Score: 0.755	MSE: -0.261
B-DTC with max_depth 11:	Score: 0.746	MSE: -0.255
B-DTC with max_depth 12:	Score: 0.763	MSE: -0.246
B-DTC with max_depth 13:	Score: 0.759	MSE: -0.248
B-DTC with max_depth 14:	Score: 0.747	MSE: -0.242
B-DTC with max_depth 15:	Score: 0.755	MSE: -0.238
B-DTC with max_depth 16:	Score: 0.755	MSE: -0.237
B-DTC with max_depth 17:	Score: 0.757	MSE: -0.250
B-DTC with max_depth 18:	Score: 0.741	MSE: -0.266
B-DTC with max_depth 19:	Score: 0.747	MSE: -0.256
B-DTC with max_depth 20:	Score: 0.753	MSE: -0.257
B-DTC with max_depth 21:	Score: 0.727	MSE: -0.273


In [279]:
regr = LogisticRegression()
regr.fit(X,y)
score = cross_val_score(regr,X,y)
mse = cross_val_score(regr,X,y,scoring="neg_mean_squared_error")
print(f"Logistic Regression:\nScore: {score.mean():.3f}\tMSE:{mse.mean():.3f}")

Logistic Regression:
Score: 0.779	MSE:-0.221


In [277]:
X.shape

(1000, 26)

In [297]:
text1 = "I hate myself, I'm going to kill myself, this day is terrible. What am I doing with my life???"
text2 = "What a beautiful day, I feel so happy!"
text3 = "It is not a lack of love, but a lack of friendship that makes unhappy marriages."
text4 = "Whoever fights monsters should see to it that in the process he does not become a monster. And if you gaze long enough into an abyss, the abyss will gaze back into you."
text5 = "God is dead. God remains dead. And we have killed him. Yet his shadow still looms. How shall we comfort ourselves, the murderers of all murderers? What was holiest and mightiest of all that the world has yet owned has bled to death under our knives; who will wipe this blood off us? What water is there for us to clean ourselves?"
text6 = "Today sure is a terrible day. My sister just died, and I lost my job. However, I will not let my depression take over, since this would left everyone i know in sorrow!"

text_ls = [text1,text2,text3,text4,text5,text6]
# I would personally say 1,0,0,?,1

data = np.array([sentiment_text(text) + get_entity_sentiment(text) for text in text_ls])

data

array([[-0.5       ,  1.10000002,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.89999998,  0.89999998,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.69999999,  0.69999999,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.2       ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       

In [294]:
regr.predict(data)

array([1, 0, 1, 0, 1])

In [296]:
boosted_dtc = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4))
boosted_dtc.fit(X,y)
boosted_dtc.predict(data)

array([1, 0, 1, 1, 1])